In [1]:
import tensorflow as tf
from tensorflow.python import pywrap_tensorflow
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

checkpoint_path = '/workspace/models/fine-tuned/bert_tf_v2_base_fp16_128_v2/model.ckpt-8144'
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
names_nv = [key for key in var_to_shape_map]
# for key in var_to_shape_map:
#     print("tensor_name: ", key)
#     print(reader.get_tensor(key))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
checkpoint_path = 'eval_ckpts/model.ckpt-315171'
# 'output/model.ckpt-0'
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
names_sp = [key for key in var_to_shape_map]

In [3]:
checkpoint_path = 'output/model.ckpt-0'
# 'eval_ckpts/model.ckpt-315171'
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
names_op = [key for key in var_to_shape_map]

In [4]:
len(names_nv), len(names_sp), len(names_op)

(603, 618, 604)

In [5]:
# names_nv

In [6]:
# names_sp

In [7]:
[name for name in names_sp if name not in names_op], [name for name in names_op if name not in names_nv]

(['fc0/weights/adam_m',
  'fc0/biases/adam_m',
  'conv2/kernel/adam_v',
  'conv2/kernel/adam_m',
  'conv2/biases/adam_v',
  'conv2/biases/adam_m',
  'conv1/kernel',
  'conv1/biases',
  'conv0/kernel/adam_v',
  'conv0/kernel/adam_m',
  'conv0/kernel',
  'conv0/biases',
  'conv2/kernel',
  'conv1/kernel/adam_m',
  'conv0/biases/adam_m',
  'conv0/biases/adam_v',
  'fc0/biases',
  'conv1/biases/adam_v',
  'conv2/biases',
  'fc0/weights',
  'fc0/biases/adam_v',
  'fc0/weights/adam_v',
  'conv1/biases/adam_m',
  'conv1/kernel/adam_v'],
 ['output_weights/adam_v',
  'output_weights/adam_m',
  'output_bias/adam_v',
  'output_bias',
  'bert/pooler/dense/kernel/adam_m',
  'output_weights',
  'output_bias/adam_m',
  'bert/pooler/dense/bias/adam_m',
  'bert/pooler/dense/bias/adam_v',
  'bert/pooler/dense/kernel/adam_v'])

In [8]:
add_ons = [name for name in names_sp if name not in names_nv and 'adam' not in name]
add_ons

['conv1/kernel',
 'conv1/biases',
 'conv0/kernel',
 'conv0/biases',
 'conv2/kernel',
 'fc0/biases',
 'conv2/biases',
 'fc0/weights']

In [9]:
[name for name in names_nv if name not in names_sp]

['cls/squad/output_weights/adam_m',
 'cls/squad/output_bias/adam_v',
 'cls/squad/output_bias/adam_m',
 'cls/squad/output_bias',
 'cls/squad/output_weights/adam_v',
 'good_steps',
 'cls/squad/output_weights',
 'bad_steps',
 'loss_scale']

In [10]:
import tensorrt as trt
import ctypes
import argparse
import numpy as np
import json
import sys
import re
import os

try:
    from tensorflow.python import pywrap_tensorflow as pyTF
except ImportError as err:
    sys.stderr.write("""Error: Failed to import tensorflow module ({})""".format(err))
    sys.exit()

"""
TensorRT Initialization
"""
TRT_LOGGER = trt.Logger(trt.Logger.INFO)

ctypes.CDLL("libnvinfer_plugin.so", mode=ctypes.RTLD_GLOBAL)
ctypes.CDLL("/workspace/TensorRT/demo/BERT/build/libcommon.so", mode=ctypes.RTLD_GLOBAL)
ctypes.CDLL("/workspace/TensorRT/demo/BERT/build/libbert_plugins.so", mode=ctypes.RTLD_GLOBAL)

trt.init_libnvinfer_plugins(TRT_LOGGER, "")
plg_registry = trt.get_plugin_registry()
qkv2_plg_creator = plg_registry.get_plugin_creator("CustomQKVToContextPluginDynamic", "1", "")
skln_plg_creator = plg_registry.get_plugin_creator("CustomSkipLayerNormPluginDynamic", "1", "")
gelu_plg_creator = plg_registry.get_plugin_creator("CustomGeluPluginDynamic", "1", "")
emln_plg_creator = plg_registry.get_plugin_creator("CustomEmbLayerNormPluginDynamic", "1", "")


"""
Attentions Keys
"""
WQ = "query_kernel"
BQ = "query_bias"
WK = "key_kernel"
BK = "key_bias"
WV = "value_kernel"
BV = "value_bias"
WQKV = "qkv_kernel"
BQKV = "qkv_bias"


"""
Transformer Keys
"""
W_AOUT = "attention_output_dense_kernel"
B_AOUT = "attention_output_dense_bias"
AOUT_LN_BETA = "attention_output_layernorm_beta"
AOUT_LN_GAMMA = "attention_output_layernorm_gamma"
W_MID = "intermediate_dense_kernel"
B_MID = "intermediate_dense_bias"
W_LOUT = "output_dense_kernel"
B_LOUT = "output_dense_bias"
LOUT_LN_BETA = "output_layernorm_beta"
LOUT_LN_GAMMA = "output_layernorm_gamma"


"""
Squad Output Keys
"""
SQD_W = "squad_output_weights"
SQD_B = "squad_output_bias"

"""
CoLA Output Keys
"""
CoLA_W = "output_weights"
CoLA_B = "output_bias"

class BertConfig:
    def __init__(self, bert_config_path):
        with open(bert_config_path, 'r') as f:
            data = json.load(f)
            self.num_attention_heads = data['num_attention_heads']
            self.hidden_size = data['hidden_size']
            self.intermediate_size = data['intermediate_size']
            self.num_hidden_layers = data['num_hidden_layers']
            self.use_fp16 = True


def set_tensor_name(tensor, prefix, name):
    tensor.name = prefix + name

def set_layer_name(layer, prefix, name, out_idx = 0):
    set_tensor_name(layer.get_output(out_idx), prefix, name)

def attention_layer_opt(prefix, config, init_dict, network, input_tensor, imask):
    """
    Add the attention layer
    """
    assert(len(input_tensor.shape) == 5)
    B, S, hidden_size, _, _ = input_tensor.shape
    num_heads = config.num_attention_heads
    head_size = int(hidden_size / num_heads)

    Wall = init_dict[prefix + WQKV]
    Ball = init_dict[prefix + BQKV]

    mult_all = network.add_fully_connected(input_tensor, 3 * hidden_size, Wall, Ball)
    set_layer_name(mult_all, prefix, "qkv_mult")

    has_mask = imask is not None

    pf_hidden_size = trt.PluginField("hidden_size", np.array([hidden_size], np.int32), trt.PluginFieldType.INT32)
    pf_num_heads = trt.PluginField("num_heads", np.array([num_heads], np.int32), trt.PluginFieldType.INT32)
    pf_has_mask = trt.PluginField("has_mask", np.array([has_mask], np.int32), trt.PluginFieldType.INT32)

    pfc = trt.PluginFieldCollection([pf_hidden_size, pf_num_heads, pf_has_mask])
    qkv2ctx_plug = qkv2_plg_creator.create_plugin("qkv2ctx", pfc)

    qkv_in = [mult_all.get_output(0), imask]
    qkv2ctx = network.add_plugin_v2(qkv_in, qkv2ctx_plug)
    set_layer_name(qkv2ctx, prefix, "context_layer")
    return qkv2ctx


def skipln(prefix, init_dict, network, input_tensor, skip):
    """
    Add the skip layer
    """
    idims = input_tensor.shape
    assert len(idims) == 5
    hidden_size = idims[2]

    pf_ld = trt.PluginField("ld", np.array([hidden_size], np.int32), trt.PluginFieldType.INT32)
    wbeta = init_dict[prefix + "beta"]
    pf_beta = trt.PluginField("beta", wbeta.numpy(), trt.PluginFieldType.FLOAT32)
    wgamma = init_dict[prefix + "gamma"]
    pf_gamma = trt.PluginField("gamma", wgamma.numpy(), trt.PluginFieldType.FLOAT32)

    pfc = trt.PluginFieldCollection([pf_ld, pf_beta, pf_gamma])
    skipln_plug = skln_plg_creator.create_plugin("skipln", pfc)

    skipln_inputs = [input_tensor, skip]
    layer = network.add_plugin_v2(skipln_inputs, skipln_plug)
    return layer


def transformer_layer_opt(prefix, config, init_dict, network, input_tensor, imask):
    """
    Add the transformer layer
    """
    idims = input_tensor.shape
    assert len(idims) == 5
    hidden_size = idims[2]

    context_transposed = attention_layer_opt(prefix + "attention_self_", config, init_dict, network, input_tensor, imask)
    attention_heads = context_transposed.get_output(0)

    W_aout = init_dict[prefix + W_AOUT]
    B_aout = init_dict[prefix + B_AOUT]
    attention_out_fc = network.add_fully_connected(attention_heads, hidden_size, W_aout, B_aout)

    skiplayer = skipln(prefix + "attention_output_layernorm_", init_dict, network, attention_out_fc.get_output(0), input_tensor)
    attention_ln = skiplayer.get_output(0)

    W_mid = init_dict[prefix + W_MID]
    B_mid = init_dict[prefix + B_MID]
    mid_dense = network.add_fully_connected(attention_ln, config.intermediate_size, W_mid, B_mid)

    mid_dense_out = mid_dense.get_output(0)

    pfc = trt.PluginFieldCollection()
    plug = gelu_plg_creator.create_plugin("gelu", pfc)

    gelu_layer = network.add_plugin_v2([mid_dense_out], plug)

    intermediate_act = gelu_layer.get_output(0)
    set_tensor_name(intermediate_act, prefix, "gelu")

    # Dense to hidden size
    W_lout = init_dict[prefix + W_LOUT]
    B_lout = init_dict[prefix + B_LOUT]

    out_dense = network.add_fully_connected(intermediate_act, hidden_size, W_lout, B_lout)
    set_layer_name(out_dense, prefix + "output_", "dense")
    out_layer = skipln(prefix + "output_layernorm_", init_dict, network, out_dense.get_output(0), attention_ln)
    out_ln = out_layer.get_output(0)

    set_tensor_name(out_ln, prefix + "output_", "reshape")

    return out_ln


def bert_model(config, init_dict, network, input_tensor, input_mask):
    """
    Create the bert model
    """
    prev_input = input_tensor
    output = []
    for layer in range(0, config.num_hidden_layers):
        ss = "l{}_".format(layer)
        prev_input = transformer_layer_opt(ss, config,  init_dict, network, prev_input, input_mask)
        output.append(prev_input)
    return prev_input, output


def squad_output(prefix, config, init_dict, network, input_tensor):
    """
    Create the squad output
    """

    idims = input_tensor.shape
    assert len(idims) == 5
    B, S, hidden_size, _, _ = idims

    W_out = init_dict[prefix + SQD_W]
    B_out = init_dict[prefix + SQD_B]

    W = network.add_constant((1, hidden_size, 2), W_out)
    dense = network.add_fully_connected(input_tensor, 2, W_out, B_out)
    set_layer_name(dense, prefix, "dense")
    return dense


def load_weights(inputbase):
    """
    Load the weights from the tensorflow checkpoint
    """
    weights_dict = dict()

    try:
        reader = pyTF.NewCheckpointReader(inputbase)
        tensor_dict = reader.get_variable_to_shape_map()

        # There might be training-related variables in the checkpoint that can be discarded
        param_names = [key for key in sorted(tensor_dict) if 'adam' not in key and 'global_step' not in key and 'pooler' not in key]
        count = len(param_names)
        TRT_LOGGER.log(TRT_LOGGER.INFO, "Found {:} entries in weight map".format(count))

        for pn in param_names:
            toks = pn.lower().split('/')
            if 'encoder' in pn:
                assert ('layer' in pn)
                l = (re.findall('\d+', pn))[0]
                outname = 'l{}_'.format(l) + '_'.join(toks[3:])
            else:
                outname = '_'.join(toks)

            tensor = reader.get_tensor(pn)
            shape = tensor.shape
            if pn.find('kernel') != -1:
                TRT_LOGGER.log(TRT_LOGGER.VERBOSE, "Transposing {}\n".format(np))
                tensor = np.transpose(tensor) # for kernel in conv layers, it's supposed to be
                                              # transposed with(3,2,0,1), but since ours is 1d
                                              # convolution, default (3,2,1,0) works fine.
         
            shape = tensor.shape
            flat_tensor = tensor.flatten()
            print(pn, shape, flat_tensor.shape)
            shape_str = '{} '.format(len(shape)) + ' '.join([str(d) for d in shape])
            weights_dict[outname] = trt.Weights(flat_tensor)

            TRT_LOGGER.log(TRT_LOGGER.VERBOSE, "Orig.name: {:}, TRT name: {:}, shape: {:}".format(pn, outname, shape_str))

        additional_dict = dict()
        for key, value in weights_dict.items():
            pos = key.find(BQ)
            if pos != -1:
                hidden_size = value.size
                prefix = key[:pos]

                Bq_ = value
                Bk_ = weights_dict[prefix + BK]
                Bv_ = weights_dict[prefix + BV]
                Wq_ = weights_dict[prefix + WQ]
                Wk_ = weights_dict[prefix + WK]
                Wv_ = weights_dict[prefix + WV]

                mat_size = hidden_size * hidden_size
                wcount = 3 * mat_size
                Wall = np.zeros(wcount, np.float32)
                bcount = 3 * hidden_size
                Ball = np.zeros(bcount, np.float32)
                Wall[0:mat_size] = Wq_.numpy()[0:mat_size]
                Wall[mat_size:2*mat_size] = Wk_.numpy()[0:mat_size]
                Wall[2*mat_size:3*mat_size] = Wv_.numpy()[0:mat_size]
                Ball[0:hidden_size] = Bq_.numpy()[0:hidden_size]
                Ball[hidden_size:2*hidden_size] = Bk_.numpy()[0:hidden_size]
                Ball[2*hidden_size:3*hidden_size] = Bv_.numpy()[0:hidden_size]

                additional_dict[prefix + WQKV] = trt.Weights(Wall)
                additional_dict[prefix + BQKV] = trt.Weights(Ball)

    except Exception as error:
        TRT_LOGGER.log(TRT_LOGGER.ERROR, str(error))

    weights_dict.update(additional_dict)
    return weights_dict


def main(inputbase, B, S, bert_path, outputbase, task):
    bert_config_path = os.path.join(bert_path, 'bert_config.json')
    TRT_LOGGER.log(TRT_LOGGER.INFO, "Using configuration file: {:}".format(bert_config_path))
    config = BertConfig(bert_config_path)

    # Load weights from checkpoint file
    init_dict = load_weights(inputbase)
    
    with trt.Builder(TRT_LOGGER) as builder:
        ty = trt.PluginFieldType.FLOAT32

        w = init_dict["bert_embeddings_layernorm_beta"]
        wbeta = trt.PluginField("bert_embeddings_layernorm_beta", w.numpy(), ty)

        w = init_dict["bert_embeddings_layernorm_gamma"]
        wgamma = trt.PluginField("bert_embeddings_layernorm_gamma", w.numpy(), ty)

        w = init_dict["bert_embeddings_word_embeddings"]
        wwordemb = trt.PluginField("bert_embeddings_word_embeddings", w.numpy(), ty)

        w = init_dict["bert_embeddings_token_type_embeddings"]
        wtokemb = trt.PluginField("bert_embeddings_token_type_embeddings", w.numpy(), ty)

        w = init_dict["bert_embeddings_position_embeddings"]
        wposemb = trt.PluginField("bert_embeddings_position_embeddings", w.numpy(), ty)

        pfc = trt.PluginFieldCollection([wbeta, wgamma, wwordemb, wtokemb, wposemb])
        fn = emln_plg_creator.create_plugin("embeddings", pfc)

        explicit_batch_flag = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
        with builder.create_network(explicit_batch_flag) as network, builder.create_builder_config() as builder_config:
            builder_config.max_workspace_size = 5000 * (1024 * 1024) # 5000 MiB
            builder_config.set_flag(trt.BuilderFlag.FP16)

            input_ids = network.add_input(name="input_ids", dtype=trt.int32, shape=(-1, S))
            segment_ids = network.add_input(name="segment_ids", dtype=trt.int32, shape=(-1, S))
            input_mask = network.add_input(name="input_mask", dtype=trt.int32, shape=(-1, S))

            def set_profile_shape(profile, batch_size):
                shape = (batch_size, S)
                profile.set_shape("input_ids", min=shape, opt=shape, max=shape)
                profile.set_shape("segment_ids", min=shape, opt=shape, max=shape)
                profile.set_shape("input_mask", min=shape, opt=shape, max=shape)

            # Specify profiles for the batch sizes we're interested in.
            # For maximum performance, we will tie each profile to exactly one shape rather than a range.
            bs1_profile = builder.create_optimization_profile()
            set_profile_shape(bs1_profile, 1)
            builder_config.add_optimization_profile(bs1_profile)

            bs_user_profile = builder.create_optimization_profile()
            set_profile_shape(bs_user_profile, B)
            builder_config.add_optimization_profile(bs_user_profile)

            bs8_profile = builder.create_optimization_profile()
            set_profile_shape(bs8_profile, 8)
            builder_config.add_optimization_profile(bs8_profile)

            # Create the network
            inputs = [input_ids, segment_ids, input_mask]
            emb_layer = network.add_plugin_v2(inputs, fn)

            embeddings = emb_layer.get_output(0)
            mask_idx = emb_layer.get_output(1)
            if task == "bert_embedding":
                bert_out, layers = bert_model(config, init_dict, network, embeddings, mask_idx)
                B, S, hidden_size, _, _ = bert_out.shape
                for i in range(len(layers)):
                    print(i, layers[i].shape)
                concat = network.add_concatenation(layers)
                reshape = network.add_shuffle(concat.get_output(0))
                reshape.reshape_dims = (B, S, len(layers), hidden_size)
                reshape.second_transpose = (0, 2, 1, 3)
                out = reshape.get_output(0)
                
            if task == "squad":
                bert_out, _ = bert_model(config, init_dict, network, embeddings, mask_idx)
                squad_logits = squad_output("cls_", config, init_dict, network, bert_out)
                out = squad_logits.get_output(0)
            elif task == "cola":
                bert_out, _ = bert_model(config, init_dict, network, embeddings, mask_idx)
                cola_logits = cola_output('cola', config, init_dict, network, bert_out)
                out = cola_logits.get_output(0)
            print(out.shape)
            network.mark_output(out)
            print("Done building the network")
            with builder.build_engine(network, builder_config) as engine:
                TRT_LOGGER.log(TRT_LOGGER.VERBOSE, "Serializing Engine...")
                serialized_engine = engine.serialize()
                TRT_LOGGER.log(TRT_LOGGER.INFO, "Saving Engine to {:}".format(outputbase))
                with open(outputbase, 'wb') as fout:
                    fout.write(serialized_engine)
                TRT_LOGGER.log(TRT_LOGGER.INFO, "Done.")

### notes: if conv layers are built using a for loop, conv_w & conv_b will be updated each iteration but its values will only be used during engine building process, so a bag is needed to hold them

In [11]:
def cola_output(prefix, config, init_dict, network, input_tensor):
    """
    Create the CoLA output
    """
    idims = input_tensor.shape
    assert len(idims) == 5
    B, S, hidden_size, _, _ = idims
  
    # add shuffle layer for reshaping and permutation 
    shuffle = network.add_shuffle(input_tensor)
    shuffle.first_transpose = (0, 2, 1, 3, 4)
    shuffle.reshape_dims = (B, hidden_size, S,1)
    input_tensor = shuffle.get_output(0)
    print(input_tensor.shape)
    # add convolution layers    
    conv_outputs = []
    bag = []
    for i in range(3):
        # add conv
        kernel_size = trt.DimsHW(3 + i, 1)
        conv_w = init_dict['conv{}_kernel'.format(i)]
        conv_b = init_dict['conv{}_biases'.format(i)]
        print(input_tensor.shape, kernel_size, conv_w.size)
        conv = network.add_convolution(
            input=input_tensor, 
            num_output_maps=100,
            kernel_shape=kernel_size,
            kernel=conv_w,
            bias=conv_b
        )
        conv.stride = (1, 1)
        conv.padding_mode=trt.PaddingMode.SAME_UPPER
        bag += [conv_w, conv_b]
        set_layer_name(conv, prefix, "conv{}".format(i))
        print("conv output shape: ", conv.get_output(0).shape)
        # add relu
        relu = network.add_activation(input=conv.get_output(0), type=trt.ActivationType.RELU)
        set_layer_name(relu, prefix, "relu{}".format(i))
        # add pooling
        pooling = network.add_pooling(
            input=relu.get_output(0), 
            type=trt.PoolingType.MAX,
            window_size=(8,1)
        )
        pooling.stride = (1, 1)
        set_layer_name(pooling, prefix, "pooling{}".format(i))
        print("Pooling output shape", pooling.get_output(0).shape)
        # for concat
        conv_outputs.append(pooling.get_output(0))
        
    concat = network.add_concatenation(inputs=conv_outputs)
    print("Concat output shape:", concat.get_output(0).shape)
    set_layer_name(concat, prefix, "concat")
    # fc layer
    dense = network.add_fully_connected(concat.get_output(0), 22, 
                                        init_dict['fc0_weights'].numpy(), init_dict['fc0_biases'].numpy())
    set_layer_name(dense, prefix, "dense")
    print("fc layer output shape: ", dense.get_output(0).shape)
    
    softmax = network.add_softmax(input=dense.get_output(0))
    print("softmax layer output shape: ", softmax.get_output(0).shape)
    return softmax

In [12]:
inputbase = 'eval_ckpts/model.ckpt-315171'# 'output/model.ckpt-0'
outputbase = 'bert_embedding_8_batch1_single_profile.engine'
B = 1
S = 8
bert_path = 'eval_ckpts'
main(inputbase, B, S, bert_path, outputbase, 'bert_embedding')

bert/embeddings/LayerNorm/beta (768,) (768,)
bert/embeddings/LayerNorm/gamma (768,) (768,)
bert/embeddings/position_embeddings (512, 768) (393216,)
bert/embeddings/token_type_embeddings (2, 768) (1536,)
bert/embeddings/word_embeddings (30522, 768) (23440896,)
bert/encoder/layer_0/attention/output/LayerNorm/beta (768,) (768,)
bert/encoder/layer_0/attention/output/LayerNorm/gamma (768,) (768,)
bert/encoder/layer_0/attention/output/dense/bias (768,) (768,)
bert/encoder/layer_0/attention/output/dense/kernel (768, 768) (589824,)
bert/encoder/layer_0/attention/self/key/bias (768,) (768,)
bert/encoder/layer_0/attention/self/key/kernel (768, 768) (589824,)
bert/encoder/layer_0/attention/self/query/bias (768,) (768,)
bert/encoder/layer_0/attention/self/query/kernel (768, 768) (589824,)
bert/encoder/layer_0/attention/self/value/bias (768,) (768,)
bert/encoder/layer_0/attention/self/value/kernel (768, 768) (589824,)
bert/encoder/layer_0/intermediate/dense/bias (3072,) (3072,)
bert/encoder/layer_0

bert/encoder/layer_8/output/dense/kernel (768, 3072) (2359296,)
bert/encoder/layer_9/attention/output/LayerNorm/beta (768,) (768,)
bert/encoder/layer_9/attention/output/LayerNorm/gamma (768,) (768,)
bert/encoder/layer_9/attention/output/dense/bias (768,) (768,)
bert/encoder/layer_9/attention/output/dense/kernel (768, 768) (589824,)
bert/encoder/layer_9/attention/self/key/bias (768,) (768,)
bert/encoder/layer_9/attention/self/key/kernel (768, 768) (589824,)
bert/encoder/layer_9/attention/self/query/bias (768,) (768,)
bert/encoder/layer_9/attention/self/query/kernel (768, 768) (589824,)
bert/encoder/layer_9/attention/self/value/bias (768,) (768,)
bert/encoder/layer_9/attention/self/value/kernel (768, 768) (589824,)
bert/encoder/layer_9/intermediate/dense/bias (3072,) (3072,)
bert/encoder/layer_9/intermediate/dense/kernel (3072, 768) (2359296,)
bert/encoder/layer_9/output/LayerNorm/beta (768,) (768,)
bert/encoder/layer_9/output/LayerNorm/gamma (768,) (768,)
bert/encoder/layer_9/output/den